In [27]:
import torch
from ganpaint import nethook, setting, renormalize, zdataset, widget
from ipywidgets import Output, Box, VBox, Button

# Load a generator, and paint random image number 20 in a widget.
G = nethook.InstrumentedModel(setting.load_proggan('church')).cuda()
z = zdataset.z_sample_for_model(G)[20][None].cuda()
V = widget.PaintWidget(image=renormalize.as_url(G(z)[0]))

# Whenever the widget changes, call "changed" to edit the layer and repaint
current_mask, current_target = [torch.zeros(1, 512, 8, 8).cuda() for _ in [1,2]]
def editrule(x, imodel, **buffers):
    global current_mask, current_target
    x = x * (1 - current_mask) + current_target * current_mask
    return x
G.edit_layer('layer4', rule=editrule)

colors = [119, 125, 157, 195, 365, 374, 418, 426, 408, 3, 297, 322, 343]
selected_units = TREE_UNITS

out = Output()

def paint(units, value, location):
    global current_mask, current_target
    paint_mask, target = [torch.zeros(current_mask.shape).cuda() for _ in [1,2]]
    paint_mask[:, units] = location
    target[:, units] = value
    new_mask = (current_mask + paint_mask).clamp(0, 1)
    alpha = paint_mask / (new_mask + 1e-10)
    new_target = current_target * (1 - alpha) + target * alpha
    current_target, current_mask = new_target, new_mask

def reset(btn):
    global current_mask
    current_mask[...] = 0.0

def select(btn):
    
    
@out.capture()
def changed(c):
    if not V.mask: return
    a = renormalize.from_url(V.mask, target='pt', size=(8,8))[0][None, None].cuda()
    paint(selected_units, 20, a)
    V.mask, V.image = '', renormalize.as_url(G(z)[0])
V.observe(changed, names='mask')

ResetButton = Button(description='Reset')
ResetButton.on_click(reset)
choices = []
for d, u in [('Tree', TREE_UNITS)]:
    btn = Button(description=d)
    btn.on_click(lambda x: select()
          ]


display(Box([V, ResetButton ]))
display(out)

Box(children=(PaintWidget(image='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAEACAIAAADTED8xAAEAAElEQVR…

Output()

In [ ]:
V
